In [1]:
import pandas as pd;
import numpy as np;
import os

In [2]:
# ~450 genuine entries, ~150 impostor entries

genuine = {};
genuine['pressToPress'] = [];
genuine['pressToRelease'] = [];
genuine['releaseToPress'] = [];
genuine['releaseToRelease'] = [];
impostor = {};
impostor['pressToPress'] = [];
impostor['pressToRelease'] = [];
impostor['releaseToPress'] = [];
impostor['releaseToRelease'] = [];

for root, dirs, files in os.walk("data/user_001", topdown=False):
    for subdir in dirs:
        try:
            if(len(open(os.path.join(root,subdir)+"/p_pr.txt").readlines())==8):
                if "genuine" in root:
                    genuine['pressToPress'].append(open(os.path.join(root,subdir)+"/p_pp.txt").readlines())
                    genuine['pressToRelease'].append(open(os.path.join(root,subdir)+"/p_pr.txt").readlines())
                    genuine['releaseToPress'].append(open(os.path.join(root,subdir)+"/p_rp.txt").readlines())
                    genuine['releaseToRelease'].append(open(os.path.join(root,subdir)+"/p_rr.txt").readlines())
                else:
                    impostor['pressToPress'].append(open(os.path.join(root,subdir)+"/p_pp.txt").readlines())
                    impostor['pressToRelease'].append(open(os.path.join(root,subdir)+"/p_pr.txt").readlines())
                    impostor['releaseToPress'].append(open(os.path.join(root,subdir)+"/p_rp.txt").readlines())
                    impostor['releaseToRelease'].append(open(os.path.join(root,subdir)+"/p_rr.txt").readlines())
#         ignore files that aren't timestampped keystroke data
        except: 
            pass;

In [3]:
genuine['master'] = [genuine['pressToPress'][k] + genuine['pressToRelease'][k] + genuine['releaseToPress'][k] + genuine['releaseToRelease'][k] for k in range(0,len(genuine['pressToPress']))]
impostor['master'] = [impostor['pressToPress'][k] + impostor['pressToRelease'][k] + impostor['releaseToPress'][k] + impostor['releaseToRelease'][k] for k in range(0,len(impostor['pressToPress']))]

In [4]:
genuine_df = pd.DataFrame(genuine['master']).astype(int)
genuine_df['user'] = 'genuine'

impostor_df = pd.DataFrame(impostor['master']).astype(int)
impostor_df['user'] = 'impostor'

In [5]:
# Balanced dataset; 100 Genuine trials vs 100 Impostor trials

dataset = pd.concat([genuine_df.iloc[:100],impostor_df],ignore_index=True)


In [20]:
pd.concat([dataset.iloc[:10],dataset.iloc[190:]])

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,user
0,64,144,144,376,215,193,73,95,191,95,...,264,143,160,48,120,376,216,192,72,genuine
1,96,136,120,568,263,216,96,98,183,71,...,336,192,181,24,144,544,240,288,72,genuine
2,120,144,168,568,232,240,120,96,192,96,...,383,215,216,48,159,552,232,312,72,genuine
3,120,144,136,408,264,208,72,96,168,72,...,304,144,192,48,160,384,264,232,48,genuine
4,120,112,144,407,233,215,72,71,159,95,...,311,144,208,48,144,384,232,240,48,genuine
5,72,144,120,424,264,216,48,74,192,72,...,287,143,190,24,168,400,216,239,72,genuine
6,145,119,168,352,240,216,64,72,167,96,...,312,160,240,48,168,328,240,240,64,genuine
7,96,120,120,400,264,208,72,98,167,96,...,303,143,165,49,143,352,264,232,48,genuine
8,144,136,144,408,256,216,72,72,184,96,...,312,168,256,48,144,384,256,240,72,genuine
9,120,139,101,673,264,239,120,139,179,100,...,378,199,160,60,120,653,251,292,60,genuine


In [6]:
y=dataset['user']
X=dataset.drop(['user'], axis=1)


In [7]:
# Split and shuffle 

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [8]:
# Linear Support Vector Classifier

from sklearn.svm import LinearSVC
svc = LinearSVC().fit(X_train,y_train)

In [9]:
# Predict on test set and evaluate performance
svc.score(X_test,y_test)

0.97014925373134331

In [10]:
svc.decision_function(X_test)

array([ -2.58827429,  -3.18619998,  -1.76830668,   4.08453047,
         2.09088851,   4.50805675,  -3.29924965,   2.7606597 ,
         8.18838852,  -2.65079664,  -1.87908057,   2.25594189,
         2.24981903,  -3.5356738 ,   4.66083308,  12.76186222,
        -1.19942531,   5.30805852,  -2.69245769,  -1.91993889,
        29.55911001,  -1.97035391,  -0.67051392,  -3.27912906,
        -1.69563762,  -2.2530433 ,  -2.2240165 ,  -2.91187066,
         6.87070923,   1.76483571,  -2.49721255,  -3.08792986,
        -2.08736695,  -1.86758825,   6.33208035,   0.82556618,
         7.34320872,   1.28396572,   0.39265512,  -3.05831851,
        -2.66347689,  11.96432114,  -1.85554688,  -1.71343817,
        25.85802497,  13.11596196,  -2.92468298,  -3.54775906,
         6.31053919,  -2.45783601,   7.63549992,   3.55411358,
        -1.33945243,   4.34262903,   7.41949276,  -4.36278864,
         2.54644866,  -2.36034247,   3.3755373 ,  -2.09251181,
        -1.75354446,  -0.54563825,  -1.93529861,  -4.79

In [11]:
# DecisionTree Classifier
from sklearn import tree
dt = tree.DecisionTreeClassifier().fit(X_train,y_train)


In [12]:
dt.score(X_test,y_test)

0.9850746268656716

In [13]:
dt.feature_importances_

array([ 0.        ,  0.        ,  0.91395548,  0.        ,  0.04272959,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.01543013,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.0278848 ,  0.        ,  0.        ,  0.        ])

In [14]:
genuine_df.iloc[:100][2].describe()

count    100.000000
mean     145.910000
std       35.081174
min       79.000000
25%      120.000000
50%      136.000000
75%      160.000000
max      288.000000
Name: 2, dtype: float64

In [15]:
impostor_df[2].describe()


count     101.000000
mean      438.811881
std       225.546923
min       221.000000
25%       294.000000
50%       362.000000
75%       512.000000
max      1232.000000
Name: 2, dtype: float64